In [ ]:
import pandas as pd

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# read in annual wheat yields data
path = '/Users/demetrayancopoulos/Desktop/SML310/final_project/gro_ylds/wheatf_mn_ia.xlsx'
y = pd.read_excel(path)


In [ ]:
# read in features data
path = '/Users/demetrayancopoulos/Desktop/SML310/final_project/finalfeat.xlsx'
x = pd.read_excel(path)
x.drop('Unnamed: 0', axis=1, inplace=True)

# normalize all features (so that each feature has equal weight in model)
x = (x-x.min())/(x.max()-x.min())


In [ ]:
from sklearn.model_selection import train_test_split
# split into training, testing and validation set
xtrain, xtest, ytrain, ytest = train_test_split(x, y['Value'], test_size = 0.25)
xtrain, xval, ytrain, yval = train_test_split(xtrain, ytrain, test_size = 0.15)

In [ ]:
# to examine performance on test set, set xtest, ytest --> xval, yval
test = 0

if test:
    xval = xtest
    yval = ytest

# Multivariate linear regression with manual feature selection

## Initialize linear regression model

In [ ]:
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

In [ ]:
# add column of constants, so that residual mean can be centered at zero
xtrainc = sm.add_constant(xtrain)

# get linear regression with all features
lr = sm.OLS(ytrain, xtrainc).fit()

## Feature selection 

Drop insignificant features (P > threshold)

https://online.stat.psu.edu/stat462/node/180/

In [ ]:
# set maximum allowed p value
pthresh = 0.01

# get maximum p value in the current linear regression model
pmax = lr.pvalues.max()

# as long as the largest p value is greater than pthresh, drop 
# the feature with largest p value
while pmax > pthresh:
    
    # drop feature with highest p value
    imax = lr.pvalues.idxmax()
    xtrainc = xtrainc.drop(imax,1)
    
    # build new model now that a feature has been dropped
    lr = sm.OLS(ytrain, xtrainc).fit()
    
    # find highest p value in the new model
    pmax = lr.pvalues.max()

Check for multicollinearity between the features using variance inflation factor

https://etav.github.io/python/vif_factor_python.html

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as viff

In [ ]:
# create dataframe to store features and their VIFs
col = {'Explanatory var':[], 'VIF':[]}
vif = pd.DataFrame(data=col)

# get initial VIF for each feature
ncol = len(xtrainc.columns)
for i in range(0, ncol):
    vif = vif.append({'Explanatory var':xtrainc.columns[i], 'VIF':viff(xtrainc.values, i)}, ignore_index=True)

# set maximum allowed vif
vifthresh = 50

# get maximum VIF to see if any features exceed threshold
vifmax = vif['VIF'].max()

# as long as max feature is above allowed threshold, drop that feature, and generate a new model
# (MODEL IMPROVES AS VIFMAX INC --> ie more features, better accuracy)
while vifmax > vifthresh:

    # drop variable with highest VIF
    imax = vif[['VIF']].idxmax()
    drp = vif['Explanatory var'][imax]
    xtrainc = xtrainc.drop(drp,1)
    
    # build new model now that a feature has been dropped
    lr = sm.OLS(ytrain, xtrainc).fit()

    # recalculate VIF for each feature
    vif = pd.DataFrame(data=col)
    ncol = xtrainc.shape[1]
    for i in range(0, ncol):
        vif = vif.append({'Explanatory var':xtrainc.columns[i], 'VIF':viff(xtrainc.values, i)}, ignore_index=True)
    
    vifmax = vif['VIF'].max()

Now that all p values are within threshold significance level, and VIF for each feature is within allowed threshold (ie. all features are linearly independent), inspect whether the residual on training set are normally distributed.

## Validation set

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
# calculate predictions from training set 
ypred_train = lr.predict(xtrainc)

# plot residuals of training predictions of yield to actual yields 
plt.hist(ytrain-ypred_train, bins = 15)
plt.suptitle('Residuals of training predictions of yield to actual yields \n for multivariate linear regression with manual feature selection')
plt.xlabel('Residuals [tonnes per ha]')
plt.ylabel('Frequency')


Examine predictions on validation set

In [ ]:
# add constant variable to center residuals around zero
xvalt = sm.add_constant(xval)

# cols remaining in the training set after feature selection
colskept = xtrainc.columns

# make a final validation x set containing only the features 
# selected based on training set feature selection
xvalc = pd.DataFrame()
for col in colskept:
    temp = xvalt.pop(col)
    xvalc = pd.concat([xvalc, temp], axis=1)

# examine predictions on validation set
ypred_val = lr.predict(xvalc)    

In [ ]:
# compare r^2 on training set and test set
r2_val = r2_score(y_true = yval, y_pred = ypred_val)
r2_train = r2_score(y_true = ytrain, y_pred = ypred_train)

# get MSE on training set and test set
mse_val = mean_squared_error(yval, ypred_val)
mse_train = mean_squared_error(ytrain, ypred_train)

print('r2_val: ', r2_val)
print('r2_train: ', r2_train)

print('MSE val: ', mse_val)
print('MSE train: ', mse_train)

print('Number of features: ', len(xtrainc.columns))

# Multivariate linear regression with variance threshold feature selection
https://towardsdatascience.com/5-feature-selection-method-from-scikit-learn-you-should-know-ed4d116e4172

## Feature selection: variance threshold

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
# set variance threshold
varthresh = 0.02

# select features based on variance threshold
selector = VarianceThreshold(varthresh)
selector.fit(xtrain)

# compile selected features for the training and validation sets
xtrainc=xtrain[xtrain.columns[selector.get_support()]]
xvalc=xval[xtrain.columns[selector.get_support()]]

# initialize linear regression model based on the training set
lr = sm.OLS(ytrain, xtrainc).fit()

## Model Analysis

In [ ]:
# calculate predictions from training set 
ypred_train = lr.predict(xtrainc)
# calculate predictions from validation set
ypred_val = lr.predict(xvalc)

# plot residuals of training predictions of yield to actual yields 
plt.hist(ytrain-ypred_train, bins = 15)
plt.suptitle('Residuals of training predictions of yield to actual yields \n for multivariate linear regression with variance threshold feature selection')
plt.xlabel('Residuals [tonnes per ha]')
plt.ylabel('Frequency')


In [ ]:
# compare r^2 on training set and test set
r2_val = r2_score(y_true = yval, y_pred = ypred_val)
r2_train = r2_score(y_true = ytrain, y_pred = ypred_train)

# get MSE on training set and test set
mse_val = mean_squared_error(yval, ypred_val)
mse_train = mean_squared_error(ytrain, ypred_train)

print('r2_val: ', r2_val)
print('r2_train: ', r2_train)

print('MSE val: ', mse_val)
print('MSE train: ', mse_train)

print('Number of features: ', len(xtrainc.columns))

# Multivariate linear regression with recursive feature elimination

## Feature selection

In [ ]:
from sklearn.feature_selection import RFE

In [ ]:
# set number of features to select
nfeat = 30

# select features using RFE
selector = RFE(estimator=LinearRegression(), n_features_to_select=nfeat, step=1)
selector.fit(xtrain, ytrain)

# get training set with selected features
xtrainc = xtrain[xtrain.columns[selector.get_support()]]
# get validation set with selected features
xvalc = xval[xtrain.columns[selector.get_support()]]

# initialize linear regression model
lr = sm.OLS(ytrain, xtrainc).fit()

## Model analysis

In [ ]:
# calculate predictions from training set 
ypred_train = lr.predict(xtrainc)
# calculate predictions from validation set
ypred_val = lr.predict(xvalc)

# plot residuals of training predictions of yield to actual yields 
plt.hist(ytrain-ypred_train, bins = 15)
plt.suptitle('Residuals of training predictions of yield to actual yields \n for multivariate linear regression with recursive feature elimination')
plt.xlabel('Residuals [tonnes per ha]')
plt.ylabel('Frequency')

In [ ]:
# compare r^2 on training set and test set
r2_val = r2_score(y_true = yval, y_pred = ypred_val)
r2_train = r2_score(y_true = ytrain, y_pred = ypred_train)

# get MSE on training set and test set
mse_val = mean_squared_error(yval, ypred_val)
mse_train = mean_squared_error(ytrain, ypred_train)

print('r2_val: ', r2_val)
print('r2_train: ', r2_train)

print('MSE val: ', mse_val)
print('MSE train: ', mse_train)
print('Number of features: ', len(xtrainc.columns))


# Multivariable linear regression with sequential feature selection

## Feature selection

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector

In [ ]:
# set number of features to select
nfeat = 30

# select features using sequential feature selection
selector = SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select = nfeat, direction = 'backward')
selector.fit(xtrain, ytrain)

# get training set with selected features
xtrainc = xtrain[xtrain.columns[selector.get_support()]]
# get validation set with selected features
xvalc = xval[xtrain.columns[selector.get_support()]]

# initialize linear regression model
lr = sm.OLS(ytrain, xtrainc).fit()

## Model analysis

In [ ]:
# calculate predictions from training set 
ypred_train = lr.predict(xtrainc)
# calculate predictions from validation set
ypred_val = lr.predict(xvalc)

# plot residuals of training predictions of yield to actual yields 
plt.hist(ytrain-ypred_train, bins = 15)
plt.suptitle('Residuals of training predictions of yield to actual yields \n for multivariate linear regression with sequential feature selection')
plt.xlabel('Residuals [tonnes per ha]')
plt.ylabel('Frequency')

In [ ]:
# compare r^2 on training set and test set
r2_val = r2_score(y_true = yval, y_pred = ypred_val)
r2_train = r2_score(y_true = ytrain, y_pred = ypred_train)

# get MSE on training set and test set
mse_val = mean_squared_error(yval, ypred_val)
mse_train = mean_squared_error(ytrain, ypred_train)

print('r2_val: ', r2_val)
print('r2_train: ', r2_train)

print('MSE val: ', mse_val)
print('MSE train: ', mse_train)

# Multivariate polynomial regression: select features with sequential feature selection (before transforming to polynomial features)
https://www.askpython.com/python/examples/polynomial-regression-in-python

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html


In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# set number of features to select
nfeat = 10

# select features using sequential feature selection
selector = SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select = nfeat, direction = 'backward')
selector.fit(xtrain, ytrain)

# get training set with selected features
xtrainc1 = xtrain[xtrain.columns[selector.get_support()]]
# get validation set with selected features
xvalc1 = xval[xtrain.columns[selector.get_support()]]

In [ ]:
# initialize polynomial model (get all polynomial features up to deg 2)
poly = PolynomialFeatures(2)

# get the new training and validation sets 
xtrainc = poly.fit_transform(xtrainc1)
xvalc = poly.fit_transform(xvalc1)

# get all the new feature names
newcols = poly.get_feature_names_out(input_features=xtrainc1.columns)

# convert new training and validation sets into dataframes
xtrainc = pd.DataFrame(xtrainc, columns = newcols)
xvalc = pd.DataFrame(xvalc, columns = newcols)

# initialize linear regression now that features contain deg 2 polynomial
lr = sm.OLS(ytrain.values, xtrainc.values).fit()

## Model analysis

In [ ]:
# calculate predictions from training set 
ypred_train = lr.predict(xtrainc)
# calculate predictions from validation set
ypred_val = lr.predict(xvalc)

# plot residuals of training predictions of yield to actual yields 
plt.hist(ytrain-ypred_train, bins = 15)
plt.suptitle('Residuals of training predictions of yield to actual yields \n for variance thresh feat selection THEN multivariate poly (deg2) regression')
plt.xlabel('Residuals [tonnes per ha]')
plt.ylabel('Frequency')

In [ ]:
# compare r^2 on training set and test set
r2_val = r2_score(y_true = yval, y_pred = ypred_val)
r2_train = r2_score(y_true = ytrain, y_pred = ypred_train)

# get MSE on training set and test set
mse_val = mean_squared_error(yval, ypred_val)
mse_train = mean_squared_error(ytrain, ypred_train)

print('r2_val: ', r2_val)
print('r2_train: ', r2_train)

print('MSE val: ', mse_val)
print('MSE train: ', mse_train)
print('Number of features: ', len(xtrainc.columns))

# Multivariate polynomial regression: select features sequential feature selection (after transformation to polynomial features)

## Initialize polynomial deg 2 features

In [ ]:
poly = PolynomialFeatures(2)

# get training and validation set values for deg 2 polynomial features
xtrainc = poly.fit_transform(xtrain.values)
xvalc = poly.fit_transform(xval.values)

# get new feature names
newcols = poly.get_feature_names_out(input_features=xtrain.columns)

# create dataframes from new training and validation sets
xtrainc = pd.DataFrame(xtrainc, columns = newcols)
xvalc = pd.DataFrame(xvalc, columns = newcols)

## Feature selection

In [ ]:
# set variance threshold
varthresh = 0.02

# select features based on variance threshold
selector = VarianceThreshold(varthresh)
selector.fit(xtrain)

# compile selected features for the training and validation sets
xtrainc=xtrain[xtrain.columns[selector.get_support()]]
xvalc=xval[xtrain.columns[selector.get_support()]]

# initialize linear regression model
lr = sm.OLS(ytrain, xtrainc).fit()

## Model analysis

In [ ]:
# calculate predictions from training set 
ypred_train = lr.predict(xtrainc)
# calculate predictions from validation set
ypred_val = lr.predict(xvalc)


# plot residuals of training predictions of yield to actual yields 
plt.hist(ytrain-ypred_train, bins = 15)
plt.suptitle('Residuals of training predictions of yield to actual yields \n for multivariate poly (deg=2) regression THEN var thresh feat selection')
plt.xlabel('Residuals [tonnes per ha]')
plt.ylabel('Frequency')

In [ ]:
# compare r^2 on training set and test set
r2_val = r2_score(y_true = yval, y_pred = ypred_val)
r2_train = r2_score(y_true = ytrain, y_pred = ypred_train)

# get MSE on training set and test set
mse_val = mean_squared_error(yval, ypred_val)
mse_train = mean_squared_error(ytrain, ypred_train)

print('r2_val: ', r2_val)
print('r2_train: ', r2_train)

print('MSE val: ', mse_val)
print('MSE train: ', mse_train)
print('Number of features: ', len(xtrainc.columns))